In [1]:
import os
import subprocess
import nibabel as nib
import numpy as np
from nilearn import plotting
import matplotlib.pyplot as plt

root_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

def min_max_normalize(input_image_path):
    img = nib.load(input_image_path)
    img_data = img.get_fdata()
    img_data_normalized = (img_data - np.min(img_data)) / (np.max(img_data) - np.min(img_data))
    return nib.Nifti1Image(img_data_normalized, img.affine)

def affine_registration(input_image_path, output_image_path, reference_image_path):
    output_prefix = os.path.splitext(os.path.splitext(output_image_path)[0])[0]

    # Run antsRegistration
    command = f"antsRegistrationSyN.sh -d 3 -f {reference_image_path} -m {input_image_path} -o {output_prefix}_ -t a"
    subprocess.run(command, shell=True)

    # Rename the output image file
    os.rename(f"{output_prefix}_Warped.nii.gz", output_image_path)

def main():
    # Change the working directory to the "data" folder
    os.chdir('../data/input')

    # Get the list of patient folders
    patient_folders = [folder for folder in os.listdir() if os.path.isdir(folder) and folder.startswith('E')]

    reference_image_path = 'path/to/reference/image.nii.gz'  # Replace with the path to your reference image

    input_image_filenames = [
        'T2w_1mm_noalign.nii.gz',
        'T1w_1mm.nii.gz',
        'ADC_deformed.nii.gz',
        'FA_deformed.nii.gz'
    ]

    for patient_folder in patient_folders:
        for input_image_filename in input_image_filenames:
            input_image_path = os.path.join(patient_folder, 'output', input_image_filename)
            output_image_path = os.path.join(patient_folder, 'normalized', input_image_filename)

            # Create the "normalized" subfolder in each patient folder if it doesn't exist
            os.makedirs(os.path.dirname(output_image_path), exist_ok=True)

            # Perform min-max normalization
            normalized_img = min_max_normalize(input_image_path)
            nib.save(normalized_img, output_image_path)

            # Perform affine registration
            affine_registration(output_image_path, output_image_path, reference_image_path)

if __name__ == "__main__":
    main()

#change back to code directory
os.chdir('..')
os.chdir('..')
os.chdir('code')

/bin/sh: antsRegistrationSyN.sh: command not found


FileNotFoundError: [Errno 2] No such file or directory: 'E03/normalized/T2w_1mm_noalign_Warped.nii.gz' -> 'E03/normalized/T2w_1mm_noalign.nii.gz'